<a href="https://colab.research.google.com/github/sreramk1/sentiment-analysis/blob/main/training_notebooks/train_sentiment_from_existing_weights_p2v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r sentiment-analysis
!git clone https://github.com/sreramk1/sentiment-analysis.git

Cloning into 'sentiment-analysis'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 217 (delta 118), reused 142 (delta 59), pack-reused 0
Receiving objects: 100% (217/217), 8.84 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [2]:
import os
import sys

sys.path.append('/content/sentiment-analysis')
os.environ['PYTHONPATH'] += ":/content/sentiment-analysis"

In [3]:
from util.jupyter_helpers import display_tf_ds

from train_validate_predict.pandas_read_ds_from_csv import PandasReadDatasetFromCSV
from sentiment_v1.preprocess_ds import TweetReviewCsvToPdDataFrame
from sentiment_v1.convert_ds_to_tensors import TweetReviewTfDataset
from sentiment_v1.model_builder import SentimentLSTMDense64Dense32Dense16Dense8Dense1
from train_validate_predict.model_predict import ModelPredict
from train_validate_predict.model_train_validate_predict import ModelTrainValidatePredict
from model.keras_sequence_model_weights import KerasSequenceModelWeights

In [4]:
def model_build_and_train(weights=None, 
                          path="/content/sentiment-analysis/third_party/dataset/airline_sentiment_analysis.csv", 
                          negative_reivew="This is a bad airline",
                          positive_review="This is a good airline"):
  pd_dataset = PandasReadDatasetFromCSV(path)
  pd_dataset.load_dataset()

  tweet_review_csv_to_pd_dataframe = TweetReviewCsvToPdDataFrame(pd_dataset)
  tweet_review_tf_ds = TweetReviewTfDataset(tweet_review_csv_to_pd_dataframe)

  model_builder = SentimentLSTMDense64Dense32Dense16Dense8Dense1(tf_ds_store=tweet_review_tf_ds, 
                                                                 layered_weights=weights)
  model_builder.build_model()
  model = model_builder.get_model()

  model_trainer = ModelTrainValidatePredict(tweet_review_tf_ds, model)
  print("negative review before training:" + str(model_trainer.predict(negative_reivew)))
  print("positive review before training:" + str(model_trainer.predict(positive_review)))

  model_trainer.train()
  model_trainer.evaluate()
  
  print("negative review after training:" + str(model_trainer.predict(negative_reivew)))
  print("positive review after training:" + str(model_trainer.predict(positive_review)))

  return model_trainer, model

In [5]:
sequence_model_weights = KerasSequenceModelWeights()
loaded_weights = sequence_model_weights.load_weights_from_file("/content/sentiment-analysis/trained_weights/weights.json")

model_builder = SentimentLSTMDense64Dense32Dense16Dense8Dense1(layered_weights=loaded_weights)
model_builder.build_model()
model = model_builder.get_model()

model_predict = ModelPredict(model)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [6]:
model_predict.predict("This is a great airline")

array([[3.2447302]], dtype=float32)

In [7]:
model_predict.predict("This is a useless airline")

array([[-2.317034]], dtype=float32)

In [8]:
model_trainer, model_trained = model_build_and_train(sequence_model_weights.get_weights())

negative review before training:[[-1.4881439]]
positive review before training:[[2.0682502]]
Epoch 1/10
145/145 [==============================] - 20s 88ms/step - loss: 0.0881 - accuracy: 0.9760 - val_loss: 0.1015 - val_accuracy: 0.9706
Epoch 2/10
145/145 [==============================] - 9s 65ms/step - loss: 0.0719 - accuracy: 0.9796
Epoch 3/10
145/145 [==============================] - 10s 66ms/step - loss: 0.0612 - accuracy: 0.9829
Epoch 4/10
145/145 [==============================] - 10s 66ms/step - loss: 0.0532 - accuracy: 0.9854
Epoch 5/10
145/145 [==============================] - 10s 66ms/step - loss: 0.0448 - accuracy: 0.9879
Epoch 6/10
145/145 [==============================] - 9s 65ms/step - loss: 0.0385 - accuracy: 0.9891
Epoch 7/10
145/145 [==============================] - 9s 65ms/step - loss: 0.0306 - accuracy: 0.9923
Epoch 8/10
145/145 [==============================] - 10s 66ms/step - loss: 0.0260 - accuracy: 0.9939
Epoch 9/10
145/145 [==============================] 

In [9]:
sequence_model_weights.set_model(model_trained)
sequence_model_weights.read_weights_from_model()
sequence_model_weights.write_weights_to_file("weights.json")